In [1]:


from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent))
import networkx as nx
import numpy as np
from matrix import are_similar


tolerance = 1e-10


In [21]:


f = open("../graphs/graph5c.g6.txt", "r")
for line in f:
    g6 = line.strip()
    G = nx.from_graph6_bytes(g6.encode())
    A = nx.to_numpy_array(G, dtype=int)
    # One representative per similarity class, per loop count (here 1 loop)
    candidates = []
    
    for i in range(5):
        for j in range(5):
            for k in range(5):
                
                B = A.copy()
                B[i, i] = 1
                B[j, j] = 1
                B[k, k] = 1
                eigenvalues = np.linalg.eigvalsh(B)
                rounded_eigenvalues = np.round(eigenvalues / tolerance) * tolerance
                unique_eigenvalues = np.unique(rounded_eigenvalues)

                if len(unique_eigenvalues) == 3:
                    is_duplicate = any(are_similar(B, C) for C in candidates)
                    if not is_duplicate:
                        candidates.append(B)
                        print(B)
                        print(unique_eigenvalues)
                        print("\n")
        
    

[[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 0 0 1]
 [1 1 1 1 1]]
[-1.56155281 -0.          2.56155281]


[[0 0 0 1 1]
 [0 1 0 0 1]
 [0 0 1 0 1]
 [1 0 0 0 1]
 [1 1 1 1 1]]
[-1.  1.  3.]


[[0 0 0 1 1]
 [0 0 0 1 1]
 [0 0 0 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
[-1.64575131 -0.          3.64575131]


[[0 0 1 0 1]
 [0 0 0 1 1]
 [1 0 0 0 1]
 [0 1 0 0 1]
 [1 1 1 1 1]]
[-1.  1.  3.]


[[0 0 1 1 1]
 [0 0 0 0 1]
 [1 0 0 1 1]
 [1 0 1 0 1]
 [1 1 1 1 1]]
[-1.          0.58578644  3.41421356]


[[1 0 1 1 1]
 [0 1 1 1 1]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 0]]
[-1.  1.  4.]


[[0 0 1 1 1]
 [0 0 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
[-1.37228132 -0.          4.37228132]


[[1 1 1 1 1]
 [1 0 1 1 1]
 [1 1 0 1 1]
 [1 1 1 0 1]
 [1 1 1 1 0]]
[-1.         -0.23606798  4.23606798]




In [17]:
T = np.array([[1,1, 1], [1, 1, 0], [1, 0, 1]])
np.linalg.eigvalsh(T)

array([-0.41421356,  1.        ,  2.41421356])